In [6]:
import nltk 
from nltk.corpus import twitter_samples   
import matplotlib.pyplot as plt           
import random  
nltk.download('stopwords')
import re                                  
import string                             
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer 
import pandas as pd
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samanthavillanueva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#nltk.download('twitter_samples')


In [7]:
# Read each file as data frames.
neutral_df = pd.read_csv("../initial_data/resources/filtered_data/neutral.csv", index_col=[0])
positive_df = pd.read_csv("../initial_data/resources/filtered_data/positive.csv", index_col=[0])
negative_df = pd.read_csv("../initial_data/resources/filtered_data/negative.csv", index_col=[0])

In [8]:
all_positive_tweets = positive_df["tweets"].tolist()
all_neutral_tweets = neutral_df["tweets"].tolist()
all_negative_tweets = negative_df["tweets"].tolist()

print('Number of positive tweets: ', len(all_positive_tweets))
print('Number of negative tweets: ', len(all_negative_tweets))
print('Number of neutral tweets: ', len(all_neutral_tweets))


print('\nThe type of all_positive_tweets is: ', type(all_positive_tweets))
print('\nThe type of all_neutral_tweets is: ', type(all_neutral_tweets))
print('\nThe type of all_negative_tweets is: ', type(all_negative_tweets))


print('The type of a tweet entry is: ', type(all_negative_tweets[0]))
print('The type of a tweet entry is: ', type(all_positive_tweets[0]))
print('The type of a tweet entry is: ', type(all_neutral_tweets[0]))


Number of positive tweets:  17893
Number of negative tweets:  7054
Number of neutral tweets:  20509

The type of all_positive_tweets is:  <class 'list'>

The type of all_neutral_tweets is:  <class 'list'>

The type of all_negative_tweets is:  <class 'list'>
The type of a tweet entry is:  <class 'str'>
The type of a tweet entry is:  <class 'str'>
The type of a tweet entry is:  <class 'str'>


In [5]:
# total_positive_words = []
# for sentence in all_positive_tweets:
#     total_positive_words.append(sentence.count(' '))
    
# total_negative_words = []
# for sentence in all_negative_tweets:
#     total_negative_words.append(sentence.count(' '))
  
# import plotly.graph_objects as go
# import numpy as np

# x0 = np.array(total_positive_words)
# x1 = np.array(total_negative_words)

# fig = go.Figure()
# fig.add_trace(go.Histogram(x=x1, name = 'Negative'))
# fig.add_trace(go.Histogram(x=x0, name = 'Positive'))

# # Overlay both histograms
# fig.update_layout(barmode='overlay')
# # Reduce opacity to see both histograms
# fig.update_traces(opacity=0.75)
# fig.show()

In [6]:
def clean_tweets_func(tweets_stem): 

    #Remove Retweets
    def remove_RT(tweets_stem):
        return re.sub(r'^RT[\s]+', '', str(tweet))

    removed_RT = []
    for tweet in tweets_stem:
        x = remove_RT(tweet)
        removed_RT.append(x)

    #Remove Hyperlinks
    def remove_HL(removed_RT):
        return re.sub(r'https?:\/\/.*[\r\n]*', '', str(tweet))

    removed_HL = []
    for tweet in removed_RT:
        x = remove_HL(tweet)
        removed_HL.append(x)

    #Remove Hashtags
    def remove_Hash(removed_HL):
        return re.sub(r'#', '', str(tweet))
    
    removed_Hash = []
    for tweet in removed_HL:
        x = remove_Hash(tweet)
        removed_Hash.append(x)

    #Remove Numbers 
    def remove_Numbers(removed_Hash):
        return re.sub(r'[0-9]', '', str(tweet))

    removed_Numbers = []
    for tweet in removed_Hash:
        x = remove_Numbers(tweet)
        removed_Numbers.append(x)

    #Remove Emojis 
    def remove_Emojis(removed_Numbers):
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', str(tweet))

    removed_Emojis = []
    for tweet in removed_Numbers:
        x = remove_Emojis(tweet)
        removed_Emojis.append(x)

    #Remove Mentions 
    def remove_Mentions(removed_Emojis):
        return re.sub("@[A-Za-z0-9_]+","", str(tweet))

    removed_Mentions = []
    for tweet in removed_Emojis:
        x = remove_Mentions(tweet)
        removed_Mentions.append(x)

    #Store finished product from above extractions in tweet2
    tweet2 = removed_Mentions

    #Tokenizer
    def Tokenizer(tweet2):
        # instantiate the tokenizer class
        tokenizer = TweetTokenizer(preserve_case=False, 
                               strip_handles=True,
                               reduce_len=True)
        return tokenizer.tokenize(tweet)

    Tokenized = []
    for tweet in tweet2:
        x = Tokenizer(tweet)
        Tokenized.append(x)

    #Import the english stop words list from NLTK
    stopwords_english = stopwords.words('english') 

    #Removing stop-words and punctuation (we may want to customize the stop-word list and/or the punctuation )
    tweets_clean = []
    for i in range(len(Tokenized)):
        row = []
        for element in Tokenized[i]: #Go through every individual word
            if (element not in stopwords_english and  # remove stopwords
            element not in string.punctuation): # remove punctuation
                row.append(element)
        tweets_clean.append(row)
        i+=1

    #Stemming 
    # Instantiate stemming class
    stemmer = PorterStemmer() 
    # Create an empty list to store the stems
    tweets_stem = [] 
    for i in range(len(tweets_clean)):
        row =[]
        for word in tweets_clean[i]:
            stem_word = stemmer.stem(word)  # stemming word
            row.append(stem_word)  # append to the list
        tweets_stem.append(row)
        i+=1

    return(tweets_stem)

In [7]:

def create_df():
    test_pos_tweets = clean_tweets_func(all_positive_tweets)
    test_neg_tweets = clean_tweets_func(all_negative_tweets)
    test_neu_tweets = clean_tweets_func(all_neutral_tweets)
    
    pos_df = pd.DataFrame({"tweet":test_pos_tweets})
    for length in range(len(pos_df)):
        pos_df["score"] = 1.0
    
    neg_df = pd.DataFrame({"tweet":test_neg_tweets})
    for length in range(len(neg_df)):
        neg_df["score"] = -1.0
        
    neu_df = pd.DataFrame({"tweet":test_neu_tweets})
    for length in range(len(neu_df)):
        neu_df["score"] = 0.0
        
    combined_df = pd.concat([pos_df,neg_df, neu_df])
    
    return combined_df

In [8]:
combined_df = create_df()

In [9]:
combined_df = combined_df.reset_index().drop(["index"],axis=1)

In [10]:
combined_df.head()

,tweet,score
0,"[tim, tebow, may, avail, wow, jerri, uc, heck,...",1.0
1,"[aga, tayo, tomorrow, ah, :), good, night, uc,...",1.0
2,"[lunch, new, lil, spot, ..., cotton, bowl, ......",1.0
3,"[snc, halloween, pr, pump, let, us, work, sund...",1.0
4,"[loui, inspir, outfit, monday, zayn, inspir, o...",1.0


In [11]:
combined_df.tail()

,tweet,score
45451,[alright],0.0
45452,"[mom, alreadi, bought, ticket, section, sat, m...",0.0
45453,"[everyon, sat, around, metlif, annoy, didnt, l...",0.0
45454,"[giant, niner, fan, would, wanna, go, sunday, ...",0.0
45455,"[mendez, told, he'd, drive, metlif, sunday, .....",0.0


In [12]:
combined_df['score'] = combined_df['score'].replace(np.nan, 0)
combined_df.tail()

,tweet,score
45451,[alright],0.0
45452,"[mom, alreadi, bought, ticket, section, sat, m...",0.0
45453,"[everyon, sat, around, metlif, annoy, didnt, l...",0.0
45454,"[giant, niner, fan, would, wanna, go, sunday, ...",0.0
45455,"[mendez, told, he'd, drive, metlif, sunday, .....",0.0


In [13]:
combined_df.to_csv('Resources/Combined_Sentiments.csv', index = False)

In [14]:
len(combined_df[combined_df["score"]==0])

20509